# Should you delay RRSP Contributions?
This notebook contains my rough calculations to answer the question: if you expect to be in a higher tax bracket in the future, does it make financial sense to delay Canadian RRSP contributions until you earn more (and thus enter a higher tax bracket)?

Essentially, this notebook is a calculator to analyze the effect of a given contribution strategy on a given financial situation.

Some important notes:
- I am not in accounting, or anything similar. So I don't know 100% how taxes work  
- This is a very simplified analysis. Things like TFSA and minimum deductions should be taken into account

# Goal
Given an arbitrary tax scheme and expected income, calculate the total wealth generated by a given RRSP contribution plan. The goal is to calculate total net wealth after tax.

# Parameters of model
- Tax rules  
- Investment return rate (interest rate)  
- Living expenses
- RRSP contribution limit rules

## Functions set by government

In [1]:
# Some important param functions
def federal_tax_constant(taxable_income, year=-1, **kwargs):
    """
    Calculates federal tax paid on income in a certain year,
    assuming that the tax laws do not change with year.
    
    Year is still a parameter, to be consistent with the rest of the notebook
    
    Info used is for 2018:
    https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html#previousyears
    """
    return _apply_tax_scheme(taxable_income, 
                             [(0, 15.), (46603, 20.5), (93208, 26.),
                              (144489, 29.), (205842, 33.)], **kwargs)

def ontario_tax_constant(taxable_income, year=-1, **kwargs):
    """
    Calculates provincial tax paid on income in a certain year,
    assuming that the tax laws do not change with year.
    
    Year is still a parameter, to be consistent with the rest of the notebook
    
    Info used is for 2018:
    https://www.canada.ca/en/revenue-agency/services/tax/individuals/frequently-asked-questions-individuals/canadian-income-tax-rates-individuals-current-previous-years.html#previousyears
    """
    return _apply_tax_scheme(taxable_income, 
                             [(0, 5.05), (42960, 9.15), (42960+42963, 11.16),
                              (42960+42963+64077, 12.16), 
                              (220000, 13.16)], **kwargs)

def total_tax_constant(taxable_income, year=-1, province='ontario', **kwargs):
    fed_tax = federal_tax_constant(taxable_income, year, **kwargs)
    prov_tax = None
    province = province.lower()
    if province == 'ontario':
        prov_tax = ontario_tax_constant(taxable_income, year=-1, **kwargs)
    else:
        raise ValueError("Province {} not supported!".format(province))
    return fed_tax + prov_tax
    
    
def _apply_tax_scheme(income, amt_percent_tuples, debug=False):
    
    # Make sure the amounts/percents are sorted in reverse order (highest tax first)
    amt_percent_tuples.sort(key=lambda t: -t[0])
    assert all(0. <= perc <= 100. for amt, perc in amt_percent_tuples)
    
    tax = 0.
    income_not_yet_taxed = income
    for amt, perc in amt_percent_tuples:
        frac = perc / 100.
        amt_in_bracket = max(income_not_yet_taxed - amt, 0.)
        tax_paid = amt_in_bracket * frac
        tax += tax_paid
        income_not_yet_taxed -= amt_in_bracket
        if debug:
            print("Bracket: >= ${:d} at {:.1f}%: pay {} on ${} in the bracket".format(amt, perc, tax_paid, amt_in_bracket))
    return tax

def rrsp_contribution_limit_constant(taxable_income, year=-1):
    return min(taxable_income * 0.18, 26230)

## Functions set by you

In [2]:
def frugal_living_expenses(year, taxable_income):
    
    # Assume 1000 per month for rent, 500 for food, 100 for entertainment
    return 12 * (1000 + 500 + 100)

def fancy_living_expenses(year, taxable_income):
    
    # Assume 2000 per month for rent, 1000 for food, 1000 for entertainment
    return 12 * (2000 + 1000 + 1000)

# Initial step: net wealth calculator
This class holds a all the variables needed to describe one's financial circumstances

In [3]:
class FinancialCircumstance:
    def __init__(self, income_function, tax_function, 
                        rrsp_contribution_limit_function, expenses_function):
        self.income_function = income_function
        self.tax_function = tax_function
        self.rrsp_contribution_limit_function = rrsp_contribution_limit_function
        self.expenses_function = expenses_function

    def evaluate_contribution_strategy(self, years, rrsp_contribution_function, interest_rate=0.05, debug=False):

        # Argument checking
        assert 0 <= interest_rate

        # Initialize counts
        cumulative_contrib_limit = 0.
        rrsp_amount = 0.
        savings_amount = 0.

        # Do the simulation
        for year in years:

            if debug:
                print("Year: {}".format(year))

            # Calculate taxable income and contribution, and apply the contribution limit!
            taxable_income = self.income_function(year)
            
             # Can you pay expenses?
            living_expenses = self.expenses_function(year, taxable_income)
            assert taxable_income >= living_expenses
            if debug:
                print("Living expenses: {}".format(living_expenses))
            
            # Get contribution limit
            contribution_limit = self.rrsp_contribution_limit_function(taxable_income, year)
            cumulative_contrib_limit += contribution_limit
            contribution_amount = rrsp_contribution_function(taxable_income, year, cumulative_contrib_limit, living_expenses)
            if debug:
                print("Taxable Income: {}".format(taxable_income))
                print("Contribution Limit: {}".format(contribution_limit))
                print("Cumulative Limit: {}".format(cumulative_contrib_limit))
                print("Contribution Amount: {}".format(contribution_amount))
            taxable_income -= contribution_amount
            cumulative_contrib_limit -= contribution_amount
            if debug:
                print("Taxable income after contribution: {}".format(taxable_income))
                print("Cumulative limit after contribution: {}".format(cumulative_contrib_limit))
            
            # Quick checks
            assert taxable_income >= 0
            assert cumulative_contrib_limit >= 0
                

            # Calculate capital gains from your savings 
            saving_capital_gains = savings_amount * interest_rate
            taxable_income += 0.5 * saving_capital_gains     
            if debug:
                print("Savings last year: {}".format(savings_amount))   
                print("Gains on savings: {}".format(saving_capital_gains))
                print("Taxable Income after savings taxes: {}".format(taxable_income))           
            savings_amount += 0.5*saving_capital_gains
            if debug:
                print("Savings amount after capital gains: {}".format(savings_amount))

            # Calculate RRSP contribution gains while you are here
            rrsp_interest = rrsp_amount * interest_rate
            if debug:
                print("RRSP interest: {}".format(rrsp_interest))
            rrsp_amount += rrsp_interest
            rrsp_amount += contribution_amount
            if debug:
                print("RRSP Amount after interest and contribution: {}".format(rrsp_amount))

            # Check final income is ok
            assert taxable_income >= 0      

            # Calculate tax
            tax_amount = self.tax_function(taxable_income, year)
            if debug:
                print("Tax paid: {}".format(tax_amount))

            # Calculate net income
            net_income = taxable_income - living_expenses - tax_amount
            if debug:
                print("Net income: {}".format(net_income))

            # Put leftover money into savings
            savings_amount += net_income
            assert savings_amount >= 0 # otherwise you are out of money
            if debug:
                print("Total wealth at end of {}: {}".format(year, rrsp_amount + savings_amount))
                print("\n\n")

        return rrsp_amount + savings_amount

## Examine some financial strategies for a sample situation
E.g: constant income of 100k

In [4]:
# No contribution
no_contrib_fs = FinancialCircumstance(income_function=lambda x : 1e5, tax_function=federal_tax_constant,
                                  rrsp_contribution_limit_function=rrsp_contribution_limit_constant, expenses_function=frugal_living_expenses)

In [5]:
no_contrib_fs.evaluate_contribution_strategy(range(2018, 2025), lambda *args: 0, interest_rate=0.05)

498834.8943858466

In [6]:
# Always contribute your limit
contrib_limit_fs = FinancialCircumstance(income_function=lambda x : 1e5, tax_function=federal_tax_constant,
                                  rrsp_contribution_limit_function=rrsp_contribution_limit_constant, expenses_function=frugal_living_expenses)
contrib_limit_fs.evaluate_contribution_strategy(range(2018, 2025),lambda *args: args[-2], interest_rate=0.05)

535763.4747640411

### Conclusion: with a constant income, it is better to contribute to your RRSP (the obvious answer)

## Examine for more complex situation

In [7]:
# Linearly increasing income, starting at 100k increasing 20k per year
increasing_income =  FinancialCircumstance(income_function=lambda year: 1e5 + 2e4 * (year - 2018), tax_function=federal_tax_constant,
                                  rrsp_contribution_limit_function=rrsp_contribution_limit_constant, expenses_function=fancy_living_expenses)


In [8]:
# No contribution
increasing_income.evaluate_contribution_strategy(range(2018, 2030), lambda *args: 0)

1593584.9881849878

In [9]:
# Do a max contribution strategy
increasing_income.evaluate_contribution_strategy(range(2018, 2030), lambda *args: args[-2])

1722714.2679225248

In [10]:
# Wait and hold strategy
def get_wait_and_hold_strategy(wait_year):
    def func(taxable_income, year, cumulative_contrib_limit, living_expenses):
        if year < wait_year:
            return 0
        
        # Contribute as much as you can
        return min(cumulative_contrib_limit, taxable_income - living_expenses - total_tax_constant(taxable_income, year))
    
    return func
increasing_income.evaluate_contribution_strategy(range(2018, 2030), get_wait_and_hold_strategy(2020))

1721300.7817437828

### If your income is high, then it seems that delaying your RRSP contributions has no benefits

# Another (very relevant) situation: poor grad student who then years a good salary

In [11]:
def grad_student_income(year):
    if year < 2025:
        return 2.5e4
    elif year < 2030:
        return 1.5e5 + 2e4 * (year - 2025)
    else:
        return 1.5e5 + 2e4 * (2030 - 2025)
    
grad_student_fc = FinancialCircumstance(income_function=grad_student_income, tax_function=federal_tax_constant,
                                  rrsp_contribution_limit_function=rrsp_contribution_limit_constant, expenses_function=frugal_living_expenses)

In [12]:
# No contrib
grad_student_fc.evaluate_contribution_strategy(range(2018, 2030), lambda *args: 0)

707423.4426740656

In [13]:
# Max contrib
grad_student_fc.evaluate_contribution_strategy(range(2018, 2030), lambda *args: min(args[-2], max(args[0]- args[-1] - 5000, 0)), debug=False)

762316.150977415

In [14]:
# Hold until you graduate
grad_student_fc.evaluate_contribution_strategy(range(2018, 2030), get_wait_and_hold_strategy(2024), interest_rate=0.05)

762572.8503032109

## It seems that for a grad student, you might be able to save a small amount of money!!!
However, it is too little to be worth it. So in general, it probably isn't worth saving your RRSP contributions